In [1]:
import tweepy
import pandas as pd
import requests
import time

In [2]:
def process_tweet(tweet):
    mentions = []
    annotations = []
    context_annotations = []
    urls = []
    hashtags = []
    retweeting = "$"
    time_str = tweet.created_at
    if tweet.entities:
        keys = tweet.entities.keys()
    else:
        keys = []
    if tweet.text[:4] == "RT @":
        retweeting = (tweet.text.split("@")[1].split(":")[0])
    if 'annotations' in keys:
        for item in tweet.entities['annotations']:
            annotations.append(item['type'] + ":" + item['normalized_text'])
    if 'mentions' in keys:
        for item in tweet.entities['mentions']:
            mentions.append(item['username'])
    if 'urls' in keys:
        for item in tweet.entities['urls']:
            urls.append(item['expanded_url'])
    for item in tweet.context_annotations:
        if 'description' in item['entity'].keys():
            context_annotations.append(item['domain']['name'] + "," + item['entity']['name'] + "," + item['entity']['description'])
        else:
            context_annotations.append(item['domain']['name'] + "," + item['entity']['name'])
    if 'hashtags' in keys:
        for item in tweet.entities['hashtags']:
            hashtags.append(item['tag'])
    return str(time_str), str(mentions), str(annotations), str(context_annotations), str(urls), str(hashtags), retweeting

In [3]:
congdf = pd.read_csv("CongressData.csv")
congdf.head()

,Congressperson,State,Party Affiliation,House or Senate,District,Race,Gender,Birthdate,Background,Twitter Handle
0,Jerry Carl,AL,R,House,1,White,M,6/17/58,Political,RepJerryCarl
1,Barry Moore,AL,R,House,2,White,M,9/26/66,Political,RepBarryMoore
2,Mike Rogers,AL,R,House,3,White,M,7/16/58,Political,RepMikeRogersAL
3,Robert Aderholt,AL,R,House,4,White,M,7/22/65,Legal,Robert_Aderholt
4,Mo Brooks,AL,R,House,5,White,M,4/29/54,Political,RepMoBrooks


In [4]:
tweetsdf = pd.DataFrame({
    'Name': [],
    'Twitter Handle':[],
    'Time':[],
    'Content':[],
    'Retweeting':[],
    'Hashtags':[],
    'Mentions':[],
    'Annotations':[],
    'URLs':[],
    'Context Annotations':[]
})

In [5]:
num_tweets = len(tweetsdf['Name'])
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAFJiYgEAAAAA8MDdMjXr%2BTe14hDs9gUFqcH9%2BQc%3DyFU2mifWLIH7JOZUCYY0KB9V5LlldlxBoy10UsAv5lboPEgRpz')

for i in range(0, len(congdf['Congressperson'])):
    if congdf['Twitter Handle'][i] != "None?":
        name = congdf['Congressperson'][i]
        print(name, len(tweetsdf['Name']))
        handle = congdf['Twitter Handle'][i]
        query = 'from:' + handle
        #try:
        tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'entities', 'lang', 'created_at'], max_results = 100)
        if tweets.data:
            print("Found",len(tweets.data),"tweets")
            for i in range(len(tweets.data)):
                tweet = tweets.data[i]
                #print(tweet.text)
                time_str, mentions, annotations, context_annotations, urls, hashtags, retweeting = process_tweet(tweet)
                tweetsdf.loc[num_tweets] = [name, handle, time_str, tweet.text, retweeting, hashtags, mentions, annotations, urls, context_annotations]
                num_tweets += 1
    #except Exception as exc:
        #    print(exc)
        time.sleep(4.5)

Jerry Carl 0
Found 12 tweets
Barry Moore 12
Found 21 tweets
Mike Rogers 33
Found 26 tweets
Robert Aderholt 59
Found 2 tweets
Mo Brooks 61
Found 10 tweets
Gary Palmer 71
Found 13 tweets
Terri Sewell 84
Found 27 tweets
Don Young 111
Found 57 tweets
Tom O'Halleran 168
Found 14 tweets
Ann Kirkpatrick 182
Found 11 tweets
Raul Grijaiva 193
Found 12 tweets
Paul Gosar 205
Found 18 tweets
Andy Biggs 223
Found 63 tweets
David Schweikert 286
Found 6 tweets
Ruben Gallego 292
Found 24 tweets
Debbie Lesko 316
Found 55 tweets
Greg Stanton 371
Found 7 tweets
Rick Crawford 378
French Hill 378
Found 20 tweets
Steve Womack 398
Found 26 tweets
Bruce Westerman 424
Found 19 tweets
Doug LaMalfa 443
Found 6 tweets
Jared Huffman 449
Found 15 tweets
John Garamendi 464
Found 28 tweets
Tom McClintock 492
Mike Thompson 492
Found 48 tweets
Doris Matsui 540
Found 16 tweets
Ami Bera 556
Found 33 tweets
Jay Obernoite 589
Found 8 tweets
Jerry McNerney 597
Found 4 tweets
Josh Harder 601
Found 17 tweets
Mark DeSaulnier 6

In [6]:
tweetsdf.to_csv("Tweets02052022.csv")

In [24]:
tweet = tweets.data[1]
for item in tweet.context_annotations:
    print("Domain:",item['domain'])
    print("Entity Keys:",item['entity'].keys())

Domain: {'id': '45', 'name': 'Brand Vertical', 'description': 'Top level entities that describe a Brands industry'}
Entity Keys: dict_keys(['id', 'name'])
Domain: {'id': '46', 'name': 'Brand Category', 'description': 'Categories within Brand Verticals that narrow down the scope of Brands'}
Entity Keys: dict_keys(['id', 'name'])
Domain: {'id': '46', 'name': 'Brand Category', 'description': 'Categories within Brand Verticals that narrow down the scope of Brands'}
Entity Keys: dict_keys(['id', 'name'])
Domain: {'id': '47', 'name': 'Brand', 'description': 'Brands and Companies'}
Entity Keys: dict_keys(['id', 'name', 'description'])
Domain: {'id': '10', 'name': 'Person', 'description': 'Named people in the world like Nelson Mandela'}
Entity Keys: dict_keys(['id', 'name', 'description'])
Domain: {'id': '35', 'name': 'Politician', 'description': 'Politicians in the world, like Joe Biden'}
Entity Keys: dict_keys(['id', 'name', 'description'])
